## Fashion MNIST Image Classification - Single Node - CPU training

**Code tested on:**

- Tensorflow==2.1.0
- Tensorflow-datasets==2.1.0
- google-cloud-storage==1.26.0
- pandas==1.0.3

**Key activities**

- Extract and process Fashion-MNIST data
- Build Tensorflow keras model 
- Training on CPU and log the metrics ( using custom Keras logger )
- Evaluate model 
- Use Tensorboard to visualize the training process. Use kubernetes port-forward to open the tensorboard running on K8S cluster
- Model save and export
- Model upload to GCS
- Metadata tracking 

### Import libraries

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
tfds.disable_progress_bar()
import logging
from datetime import datetime
logger = tf.get_logger()
logging.basicConfig(
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt="%Y-%m-%dT%H:%M:%SZ",
    level=logging.INFO)
print('Tensorflow-version: {0}'.format(tf.__version__))

In [ ]:
# clear the logs
!rm -rf logs/

### Data extraction & processing 

In [ ]:
# prepare data
def prepare_data(batch_size=64, shuffle_size=1000):

    def scale(image, label):
        image = tf.cast(image, tf.float32)
        image /= 255
        return image, label
    
    # Split the training set into 80% and 20% for training and validation
    train_validation_split = tfds.Split.TRAIN.subsplit([8, 2])
    ((train_data, validation_data), test_data),info = tfds.load(name="fashion_mnist:1.0.0", 
                                                         split=(train_validation_split, tfds.Split.TEST),
                                                         as_supervised=True, with_info=True)

    
    print("Training data count : ", int(info.splits['train'].num_examples * 0.8))
    print("Validation data count : ", int(info.splits['train'].num_examples * 0.2))
    print("Test data count : ", int(info.splits['test'].num_examples))


    # create dataset to be used for training process
    train_dataset = train_data.map(scale).shuffle(shuffle_size).batch(batch_size).repeat().prefetch(tf.data.experimental.AUTOTUNE)
    val_dataset = validation_data.map(scale).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    test_dataset = test_data.map(scale).batch(batch_size)
    
    return train_dataset, val_dataset, test_dataset

### Build Model 

In [ ]:
def build_model(learning_rate=0.001):
    # define model architecture
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(28, 28, 1), name='x'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
      ])
    # compile model with loss, optimizer and accuracy 
    model.compile(
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        metrics=['accuracy'])
    return model

### Model Callback

In [ ]:
def get_callbacks():
    # callbacks 
    # folder to store current training logs
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    class customLog(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            logging.info('epoch: {}'.format(epoch + 1))
            logging.info('loss={}'.format(logs['loss']))
            logging.info('accuracy={}'.format(logs['accuracy']))
            logging.info('val_accuracy={}'.format(logs['val_accuracy']))
    callbacks = [
        tf.keras.callbacks.TensorBoard(logdir),
        customLog()
    ]
    return callbacks

### CPU Training

In [ ]:
# Data extraction and processing
# set variables
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset, val_dataset, test_dataset = prepare_data(batch_size=BATCH_SIZE, shuffle_size=BUFFER_SIZE)

# Build Model
TF_LEARNING_RATE = 0.001
model = build_model(learning_rate=TF_LEARNING_RATE)
model.summary()

# train model
TF_EPOCHS=10
TF_STEPS_PER_EPOCHS=3
#TF_STEPS_PER_EPOCHS = int(np.ceil(48000 / float(BATCH_SIZE)))  # number of training samples / batch size

model.fit(train_dataset, 
          epochs=TF_EPOCHS,
          steps_per_epoch=3,
          validation_data=val_dataset,
          callbacks=get_callbacks())

In [ ]:
# evaluate model
result = model.evaluate(test_dataset, steps=1)
loss = result[0]
accuracy = result[1]
print("loss : {0} accuracy : {1}".format(loss, accuracy))

#### Tensorboard
Note : If you want to use Tensorboard : use tensorboard command and run from the terminal ( from notebook home page -> new -> terminal )

```
tensorboard --logdir=/home/jovyan/logs/ --bind_all
```
if you are running inside a **container** you can use **port-mapping**. if you are running inside **kubernetes pod**, then use the pod **port-forward feature** on the port 6006 (default for tensorboard, change it as per the tensorboard command output ). When a notebook is created, a pod with name <NOTEBOOK_NAME>-0 is created in the users namespace. So you can use the port-forward to access tensorboard. 

```
kubectl port-forward -n <NAMESPACE> <NOTEBOOK_NAME>-0 6006:6006
```


### Model save and export

In [ ]:
# save keras model 
model.save("model.h5")

In [ ]:
# create model from saved model
model_loaded = tf.keras.models.load_model('model.h5')

In [ ]:
# generate summary
model_loaded.summary()

In [ ]:
# export model
tf.saved_model.save(model_loaded, "export/")

In [ ]:
!ls export/

In [ ]:
!saved_model_cli show --dir export/ --tag_set serve --signature_def serving_default

#### Save on GCS

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import glob
from google.cloud import storage
import re
import logging

_GCS_PREFIX = "gs://"


class Storage(object):
    @staticmethod
    def upload(uri: str, out_dir: str = None) -> str:
        logging.info("Copying contents from %s to %s", uri, out_dir)

        if out_dir.startswith(_GCS_PREFIX):
            Storage._upload_gcs(uri, out_dir)
        else:
            raise Exception("Cannot recognize storage type for " + uri +
                            "\n'%s' are the current available storage type." %
                            (_GCS_PREFIX))

        logging.info("Successfully copied %s to %s", uri, out_dir)
        return out_dir
    
    @staticmethod
    def _upload_gcs(uri, out_dir: str):
        try:
            storage_client = storage.Client()
        except exceptions.DefaultCredentialsError:
            storage_client = storage.Client.create_anonymous_client()
        
        bucket_args = out_dir.replace(_GCS_PREFIX, "", 1).split("/", 1)
        bucket_name = bucket_args[0]
        gcs_path = bucket_args[1] if len(bucket_args) > 1 else ""
        bucket = storage_client.bucket(bucket_name)
        Storage.upload_local_directory_to_gcs(uri,bucket, gcs_path)
    
    @staticmethod
    def upload_local_directory_to_gcs(local_path, bucket, gcs_path):
        assert os.path.isdir(local_path)
        for local_file in glob.glob(local_path + '/**'):
            if not os.path.isfile(local_file):
                Storage.upload_local_directory_to_gcs(local_file, bucket, gcs_path + "/" + os.path.basename(local_file))
            else:
                remote_path = os.path.join(gcs_path, local_file[1 + len(local_path):])
                blob = bucket.blob(remote_path)
                blob.upload_from_filename(local_file)
  

In [ ]:
!echo ${GOOGLE_APPLICATION_CREDENTIALS}

In [ ]:
%%bash
# get project ID 
PROJECT_ID=$(gcloud config get-value core/project)
# create bucket 
BUCKET=${PROJECT_ID}-fashion-mnist
# delete the bucket if exists before - CAUTION: THIS WILL REMOVE THE BUCKET
gsutil rm -r gs://$BUCKET
# create the bucket
gsutil mb gs://$BUCKET/

In [ ]:
project_id=!gcloud config get-value core/project
bucket="gs://{0}-fashion-mnist/export/001".format(project_id[0])
print(bucket)

In [ ]:
Storage.upload("export",bucket)

### Metadata Tracking

In [ ]:
#pip install --user kubeflow-metadata==0.3.1

In [ ]:
from kubeflow.metadata import metadata
import pandas
from datetime import datetime
from uuid import uuid4

#### Metadata store details

In [ ]:
METADATA_STORE_HOST = "metadata-grpc-service.kubeflow" # default DNS of Kubeflow Metadata gRPC serivce.
METADATA_STORE_PORT = 8080

#### Create workspace

In [ ]:
workspace = metadata.Workspace(
    # Connect to metadata service in namespace kubeflow in k8s cluster.
    store=metadata.Store(grpc_host=METADATA_STORE_HOST, grpc_port=METADATA_STORE_PORT),
    name="workspace_abhi",
    description="workspace for fashion-mnist",
    labels={"name": "user-1"})

#### Create run

In [ ]:
run = metadata.Run(
    workspace=workspace,
    name="run-" + datetime.utcnow().isoformat("T") ,
    description="deep learning based model",
)

#### create execution

In [ ]:
exec = metadata.Execution(
    name = "execution" + datetime.utcnow().isoformat("T") ,
    workspace=workspace,
    run=run,
    description="convolutional network with 32 filters",
)
print("An execution was created with id %s" % exec.id)

#### model metadata

In [ ]:
model_version = "model_version_" + str(uuid4())
model = exec.log_output(
    metadata.Model(
            name="Fashion-MNIST",
            description="model to recognize fashion items",
            owner="test@kubeflow.org",
            uri="gcs://my-bucket/fashion-mnist",
            model_type="convolutional neural network",
            training_framework={
                "name": "tensorflow",
                "version": "v2.1.0"
            },
            hyperparameters={
                "learning_rate": 0.001,
                "layers": [32, 64, 10],
                "early_stop": True
            },
            version=model_version,
            labels={"mylabel": "l1"}))
print(model)
print("\nModel id is {0.id} and version is {0.version}".format(model))

#### Dataset metadata

In [ ]:
date_set_version = "data_set_version_" + str(uuid4())
data_set = exec.log_input(
        metadata.DataSet(
            description="fashion-mnist dataset",
            name="fashion-mnist-dump",
            owner="owner@my-company.org",
            uri="gcs://my-bucket/fashion-mnist",
            version=date_set_version,
            query="SELECT * FROM fashion-mnist"))
print("Data set id is {0.id} with version '{0.version}'".format(data_set))

#### Metric metadata

In [ ]:
metrics = exec.log_output(
    metadata.Metrics(
            name="Fashion-MNIST-evaluation",
            description="validating the Fashion-MNIST model",
            owner="someone@kubeflow.org",
            uri="gcs://my-bucket/fashion-mnist-eval.csv",
            data_set_id=str(data_set.id),
            model_id=str(model.id),
            metrics_type=metadata.Metrics.VALIDATION,
            values={"loss": str(loss), "accuracy": str(accuracy)},
            labels={"mylabel": "l1"}))
print("Metrics id is %s" % metrics.id)

#### List metadata

In [ ]:
pandas.DataFrame.from_dict(workspace.list(metadata.Model.ARTIFACT_TYPE_NAME))

In [ ]:
pandas.DataFrame.from_dict(workspace.list(metadata.Metrics.ARTIFACT_TYPE_NAME))